[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prokaj/elte-python/blob/main/8-gyakorlat.ipynb)

### Conway Game of Life

The Game of Life is a cellular automaton created by mathematician John Conway in 1970. The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:

- **Overpopulation**: if a living cell is surrounded by more than three living cells, it dies.
- **Stasis**: if a living cell is surrounded by two or three living cells, it survives.
- **Underpopulation**: if a living cell is surrounded by fewer than two living cells, it dies.
- **Reproduction**: if a dead cell is surrounded by exactly three cells, it becomes a live cell.


Írjunk egy osztályt a játékhoz, pl. az `__init__` metódus hozzalétre a megadott konfigurációnak megfelelő objektumot. Legyen egy `step` metódus, ami a rendszert a következő állapotába viszi és az `__str__` metódus pedig
valahogy ábrázolja az aktuális állapotot. 

Tegyük fel, hogy a rács amin a rendszer él, egy $n\times n$-es rács, ahol mindkét irányban ciklikusan körbemegyünk,
azaz a csúcsokat modulo $n$ tekintjük.


In [ ]:
class ConwayGoL:
    def __init__(self, state, n):
        self.state = list(state)
        self.n = n
    def step(self):
        return self

    
    def __repr__(self):
        return f"{type(self).__name__}({self.state}, {self.n})"

In [ ]:
import random

init_state = [] ## ???
conway = ConwayGoL(init_state, 0)
conway.show()
conway.step()
conway.show()

A játék állapotának leírásához egy $n\times n$ rács minden pontjáról tudni kell, hogy foglalt-e vagy sem.

```
n = 11
state = [0]*(n*n)
```

Véletlenszerű kezdeti állapot:
```
state = [random.randint(0,1) for _ in range(n*n)]
```

In [ ]:
def random_state(n, p):
    return [int(random.random()<p) for _ in range(n*n)]

state = random_state(11, 0.2)
print(state)

Szebb megjelenítés?

In [ ]:
def as_matrix(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

print('\n'.join(''.join(map(str, line)) for line in as_matrix(state, n)))


In [ ]:
symbols = "\u2b1c\u2b1b"
print('\n'.join(''.join(symbols[x] for x in line) for line in as_matrix(state, n)))


In [ ]:
import matplotlib.pyplot as plt 

img = plt.matshow(as_matrix(state, n), cmap="viridis")
img.axes.axis(False)
for pos in range(0,12):
    img.axes.axhline(y=pos-0.5, color="white")
    img.axes.axvline(x=pos-0.5, color="white")
img


In [ ]:

def cgol_show(self):
    symbols = "\u2b1c\u2b1b"
    state = as_matrix([symbols[x] for x in self.state], self.n)
    return '\n'.join(''.join(line) for line in state)

ConwayGoL.__str__ = cgol_show
ConwayGoL.show = cgol_show

    

In [ ]:
conway = ConwayGoL(state, 11)
print(conway)

A `step` metódushoz ki kellene számolni egy adott csúcs foglalt szomszédainak számát `cnt`. Ha ez kész,
akkor az $i$ csúcs új állapota:

$$
    \text{state}_{t+1}[i]=
    \begin{cases}
    1 &\text{Ha $\text{cnt}[i]\in\{2,3\}$ és $\text{state}_t[i]=1$}\\
    1 &\text{Ha $\text{cnt}[i]\in\{3\}$ és $\text{state}_t[i]=0$}\\
    0 &\text{különben}
    \end{cases}
$$

In [ ]:
def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

In [ ]:
import ipytest 
ipytest.autoconfig()

In [ ]:
%%ipytest

def test_newstate():
    res = [0]*9
    res[2] = 1
    res[3] = 1
    assert newstate([1]*9, list(range(9))) == res
    res = [0]*9
    res[3] = 1    
    assert newstate([0]*9, list(range(9))) == res


In [ ]:
def idx(i, d_row, d_col, n):
    i, j = divmod(i, n)
    i = (i+d_row) % n
    j = (j+d_col) % n
    return i*n+j

def count_neighbors(state, n):
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 
    lens = len(state)
    return [sum(state[idx(i,di,dj,n)] for di, dj in strides) for i in range(lens)]

def cgol_step(self):
    count = count_neighbors(self.state, self.n)
    self.state = newstate(self.state, count)
    return self
    
ConwayGoL.step = cgol_step

In [ ]:
@classmethod
def cgol_from_random_state(cls, n, p):
    return cls(random_state(n, p), n)

ConwayGoL.from_random_state=cgol_from_random_state

In [ ]:
conway = ConwayGoL.from_random_state(4, 0.3)
print(conway)
print(as_matrix(count_neighbors(conway.state, conway.n), 4))

In [ ]:
conway = ConwayGoL.from_random_state(5, 0.5)
print(conway)
print("-"*20)
print(conway.step())

Tudunk-e valami animációszerűt készíteni? Jupyter notebook-ban pl. a következő képpen lehet: 

In [ ]:
from ipywidgets import Output
from time import sleep

out = Output()
display(out)
conway = ConwayGoL.from_random_state(n=31, p=0.2)

for i in range(100):
    out.clear_output(True)
    with out:
        print(f"After {i} steps:\n{conway}")  
    sleep(0.2)
    conway.step()
    if conway.is_empty():
        break


Ha parancssorból dolgozunk, akkor valami ilyesmit lehetne tenni

In [ ]:
%%writefile conway.py

import random

def as_matrix(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

def random_state(n, p):
    return [int(random.random()<p) for _ in range(n*n)]

def idx(i, d_row, d_col, n):
    i, j = divmod(i, n)
    i = (i+d_row) % n
    j = (j+d_col) % n
    return i*n+j

def count_neighbors(state, n):
    lens = len(state)
    strides = [(0,-1), (0, 1), (1,-1), (1,0), (1,1), (-1,-1), (-1,0), (-1,1)] 

    return [sum(state[idx(i, di, dj, n)] for di, dj in strides) for i in range(lens)]

def newstate(state, count):
    return [int((c==3)|((c==2) & (s==1))) for s, c in  zip(state, count)]

class ConwayGoL:
    symbols = "\u2b1c\u2b1b"
    
    def __init__(self, state, n):
        self.state = list(state)
        self.n = n
        assert len(state) == n*n

    def step(self):
        count = count_neighbors(self.state, self.n)
        self.state = newstate(self.state, count)
        return self

    def show(self):
        return str(self)
    
    def __str__(self): 
        symbols = self.symbols
        state = as_matrix([symbols[x] for x in self.state], self.n)
        return '\n'.join(''.join(line) for line in state)
        
    def __repr__(self):
        return f"{type(self).__name__}({self.state}, {self.n})"
    
    @classmethod
    def from_random_state(cls, n, p):
        return cls(random_state(n, p), n)

    def is_empty(self):
        return not sum(self.state)

if __name__ == "__main__":
    from time import sleep
    n = 11
    p = 0.2
    conway = ConwayGoL.from_random_state(n, p)
    back_step_string = f"{chr(27)}[{n+1}A"
    nsteps = 10
    for i in range(nsteps+1):
        print(f"after {i} step:")
        print(conway)  
        sleep(0.2)
        conway.step()
        if conway.is_empty():
            break
        if i < nsteps:
            print(back_step_string, end="")
        


Ha valamit már megírtunk és szeretnénk használni, `import`-tal elérhető. Pl.

In [ ]:
import conway
ConwayGoL?

## Mi történik a `with` statement alkalmazásakor?

### Mit lehet a with mögé írni?

Hasonlóan a `for`-hoz szinte bármit, aminek van két metódusa:

- `__enter__`
- `__exit__`


    with obj as x:
       do_something with x

Itt az `obj.__enter__()` hívás eredménye lesz az x értéke és a block végén **GARANTÁLTAN** végrehajtódik az `obj.__exit__(...)` hívás.
Az `__enter__` metódus végezheti az előkészítést, az `__exit__` a takarítást!

#### Kell-e nekünk ezeket a metódusokat közvetlenül implementálni?

Valójában nem. Elegendő egy generátor függvényt megírni:

    import time
    
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
            

Így még csak egy generátort kapunk ami egyszer visszaad semmit (`None`) majd jelzi, hogy vége van a sorozatnak. 

Próbáljuk ki `for`-ral

In [ ]:
import time
    
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")
        
for x in timer():
    print(x)

`with`-del hibát kapunk

In [ ]:
with timer() as x:
    print(x)

A `contextlib` module `contextmanager` függvénye generátorból `contextmanager`-t készít.

In [ ]:
from contextlib import contextmanager
timer2 = contextmanager(timer)
with timer2() as x:
    print(x)

Ha egy függvény akarunk alkalmazni egy függvényre, hogy azt átalakítsuk, de ugyanaz maradjon a neve akkor a python `@` szintakszist használja. Ilyenkor ,,dekoráljuk'' a függvényt. A `contextmanager` egy példa **dekorátor**ra.

    @contextmanager
    def timer():
        try:
            start = time.time()
            yield

        finally:
            runtime = time.time() - start
            print(f"run time: {runtime:3f}")
    
Példaként írjunk egy olyan contextmanager-t, ami jelzi nekünk, hogy mikor milyen hívás történik.

In [ ]:
@contextmanager
def print_whats_going_on(x):
    print("try blokk előtt")
    try:
        print("yield előtt")
        yield x
        print("yield után")
    except:
        print("except ág")
    finally:
        print("finally ág")
    print("try blokk után")

In [ ]:
with print_whats_going_on("hello") as x:
    print(x)
print("with után")

print("-"*50)

with print_whats_going_on("hello") as x:
    print(x)
    raise ValueError
print("with után")


In [ ]:
@contextmanager
def timer():
    try:
        start = time.time()
        yield

    finally:
        runtime = time.time() - start
        print(f"run time: {runtime:3f}")

Előadáson a `property` dekorátor szerepelt.

A `ConwayGoL` példában utólag adtunk metódusokat az osztályunkhoz. Ezt is megtehettük volna dekorátorral.  

In [ ]:
def conway_method(f):
    setattr(ConwayGoL, f.__name__, f)
    return f


@conway_method
def dummy_method(self):
    print("this is a message from the new method!")

c = ConwayGoL([], 0)
c.dummy_method()

Azt is megtehettük volna, hogy a osztály nincs beleégetve a kódba.

In [ ]:
def new_method(cls):
    def decorator(f):
        setattr(cls, f.__name__, f)
        return f 
    return decorator

@new_method(ConwayGoL)
def dummy_method(self):
    print("Note that the old value of dummy_method is overwritten!")

In [ ]:
c.dummy_method()